<a href="https://colab.research.google.com/github/abar-1/SDR-ML-Project/blob/main/QAMReceiverV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Function to convert binary data to QAM16 constellations

In [ ]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


16QAM Constellation Modulator

In [ ]:
import numpy as np
"""
Used https://dsplog.com/2008/06/01/binary-to-gray-code-for-16qam/ for mappings. When tested, the function works and
the binary is correctly mapped to its correspondingcomplex number based on the constellation. To check, uncomment
the last line in the cell which prints a test run of the function.
"""

def modulator(binary_data, M):
    k = int(np.log2(M))

    # defining the real and imaginary PAM constellation for 16-QAM
    alphaRe = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)
    alphaIm = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)

    # taking b0b1 for real
    ipDecRe = np.array([int(''.join(map(str, b[:k//2])), 2) for b in binary_data])
    ipGrayDecRe = ipDecRe ^ (ipDecRe >> 1)

    # taking b2b3 for imaginary
    ipDecIm = np.array([int(''.join(map(str, b[k//2:])), 2) for b in binary_data])
    ipGrayDecIm = ipDecIm ^ (ipDecIm >> 1)

    # mapping the Gray coded symbols into constellation
    modRe = alphaRe[ipGrayDecRe]
    modIm = alphaIm[ipGrayDecIm]

    # complex constellation
    mod = modRe + 1j * modIm

    return mod

def generate_qam_symbols(M=16, num_symbols=1000):
    # 4 bits per symbol
    k = int(np.log2(M))

    # Generate random binary data
    random_bits = np.random.randint(0, 2, num_symbols * k)

    # Reshape to match modulator input
    binary_data = random_bits.reshape((-1, k))

    # Modulate Binary Data
    modulated_symbols = modulator(binary_data, M)

    return modulated_symbols, binary_data
#print(generate_qam_symbols(16,5))

Adding noise to 16QAM signals

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def add_awgn(qam_symbols, snr_db):
    """
    Adds Additive White Gaussian Noise (AWGN) to QAM symbols.

    Parameters:
        qam_symbols (numpy array): The transmitted QAM symbols (complex numbers).
        snr_db (float): Signal-to-noise ratio in dB.

    Returns:
        numpy array: Noisy QAM symbols.
    """
    # Calculate signal power
    signal_power = np.mean(np.abs(qam_symbols) ** 2)

    # Compute noise power based on SNR (convert dB to linear scale)
    noise_power = signal_power / (10 ** (snr_db / 10))

    # Generate AWGN noise
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*qam_symbols.shape) + 1j * np.random.randn(*qam_symbols.shape))

    # Add noise to the symbols
    noisy_qam_symbols = qam_symbols + noise
    return noisy_qam_symbols


# Generate 16QAM symbols
M = 16
num_symbols = 1000
qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)

#Signal to noise ratio in dB
#High signal to noise ratio is good, low is bad
snr_db = 15


# Add noise to QAM symbols
noisy_qam = add_awgn(qam_symbols, snr_db)

df = pd.DataFrame({'features':original_binary.tolist(), 'target':noisy_qam.tolist()})
df.reset_index(inplace=True)

df.drop(['index'],axis=1,inplace=True)

#Getting data of different noise levels (data augmentation to prevent overfitting)
for i in range(15, 55, 5):
  M = 16
  num_symbols = 2000
  qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)
  snr_db = i
  noisy_qam = add_awgn(qam_symbols, snr_db)
  df1 = pd.DataFrame({'features':original_binary.tolist(), 'target':noisy_qam.tolist()})
  df1.reset_index(inplace=True)
  df1.drop(['index'],axis=1,inplace=True)
  df = pd.concat([df, df1])

  #Plot Constellations (Before and After Noise)
  # plt.figure(figsize=(10,5))
  # plt.subplot(1,2,2)
  # plt.scatter(noisy_qam.real, noisy_qam.imag, alpha=0.5, label="Noisy")
  # plt.title(f"Signal to Noise Ratio = {snr_db} dB)")
  # plt.xlabel("In-phase (I)")
  # plt.ylabel("Quadrature (Q)")
  # plt.grid()

  #plt.show()
df = df.rename(columns={'features': 'binary','target':'complex'})
#To save as CSV
df.to_csv('dataWithNoise.csv', index = False)




Build Neural Network

X = Complex

y = Binary

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
from sklearn.metrics import mean_squared_error, precision_score,recall_score

#Target variable (binary data)
y = df['binary'].values
# Convert each element of y to a NumPy array
y = np.array([np.array(yi) for yi in y])
#Reshape to 4 bits/symbol (-1 means numpy will figure out how many rows are needed)
y = y.reshape(-1, 4)
y = y.astype(int)

# Manually convert 4-bit binary to integer (0-15)
y = np.array([int("".join(str(bit) for bit in row), 2) for row in y])

# One-hot encoding for 16 classes
y = to_categorical(y, num_classes=16)


# Features (complex data)
X = df['complex'].values
X = np.array([np.array(xi) for xi in X])

# Reshape to (n_samples, 2) for real and imaginary parts
X = [[x.real, x.imag] for x in X]
X = np.array(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,),
          kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu',
          kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    # 16 neurons for 16 different possibilities (0000 - 1111)
    Dense(16, activation='softmax')
])

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []
y = np.round(y)
accuracies = [] #list to store the accuracy per fold
for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    y_train_norm = y_train / 15
    y_test_norm = y_test / 15

    earlyStopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        min_delta=0.001)

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    history = model.fit(X_train,
                        y_train,
                        epochs=20,
                        batch_size=32,
                        validation_split=0.2,
                        verbose=1,
                        callbacks=[earlyStopping])
    val_loss, val_acc = model.evaluate(X_test, y_test, verbose=0)
    cv_scores.append(val_acc)


# print(f"Cross-validation scores: {cv_scores}")
# print(f"Mean CV Accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")



Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


340/340 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.0778 - mae: 0.0907 - val_loss: 0.0281 - val_mae: 0.0583
Epoch 2/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0201 - mae: 0.0334 - val_loss: 0.0054 - val_mae: 0.0090
Epoch 3/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0096 - mae: 0.0191 - val_loss: 0.0029 - val_mae: 0.0031
Epoch 4/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0065 - mae: 0.0138 - val_loss: 0.0022 - val_mae: 0.0014
Epoch 5/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0053 - mae: 0.0113 - val_loss: 0.0018 - val_mae: 7.4376e-04
Epoch 6/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0054 - mae: 0.0109 - val_loss: 0.0019 - val_mae: 7.5059e-04
Epoch 7/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0046 - mae: 0.0090 - val_loss: 0.0017 - val_mae: 6.3484e-04
Epoch 8/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0045 - mae: 0.0084 - val_loss: 0.0017 - val_mae: 2.6971e-04
Epoch 9/20
340/340 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/st

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [11]:
# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
accuracy = 100 - (mse / np.mean(y_test)) * 100

# Convert probabilities to class predictions
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

precision = precision_score(y_test_classes, y_pred_classes, average='weighted') # changed average to weighted
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')  # changed average to weighted

print("MSE:", mse)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MSE: 0.0005557986034798999
Accuracy: 99.11072223443216
Precision: 0.9944384451775025
Recall: 0.9944117647058823


In [6]:
import numpy as np

X_pred = np.array([[2.5, -3], [1, -3], [3, -1], [-1, 3]])  # Convert X_pred to a NumPy array

# Assuming model.predict(X_pred) gives some output
temp = model.predict(X_pred)

predictions = []

# Find the index of the max value in each prediction
for x in range(len(temp)):
    max_val = max(temp[x])
    predictions.append(np.where(temp[x] == max_val)[0][0])
print("Complex Input: ", X_pred)


binarylist = []
for num in predictions:
    binary = ""
    if num == 0:
        binary = "0000"
    else:
        while num > 0:
            binary = str(num % 2) + binary
            num //= 2
    if(len(binary)) < 4:
        binary += "0"
    binarylist.append(binary)
print("Model Predictions: ", binarylist)
comlex = ["(-3, -3)","(-3,-1)","(-3, 1)","(-3, 3)","(-1, -3)","(-1, -1)","(-1, 1)","(-1, 3)","(1, -3)","(1, -1)","(1, 1)","(1, 3)","(3, -3)","(3, -1)","(3, 1","(3, 3)"]
bin = ["0000","0001","0011","0010","0100","0101","0111","0110","1100","1101","1111","1110","1000","1001","1011","1010"]

#Making df to help test predictions
compare = pd.DataFrame(columns=[comlex, bin])
compare




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Complex Input:  [[ 2.5 -3. ]
 [ 1.  -3. ]
 [ 3.  -1. ]
 [-1.   3. ]]
Model Predictions:  ['1000', '1100', '1001', '1100']


,"(-3, -3)","(-3,-1)","(-3, 1)","(-3, 3)","(-1, -3)","(-1, -1)","(-1, 1)","(-1, 3)","(1, -3)","(1, -1)","(1, 1)","(1, 3)","(3, -3)","(3, -1)","(3, 1","(3, 3)"
,0000,0001,0011,0010,0100,0101,0111,0110,1100,1101,1111,1110,1000,1001,1011,1010
